In [3]:
import tensorflow as tf
"""
This jupyter notebook provides an example of using Keras to complete
some of the tasks in Project 1.
"""

print(tf.keras.__version__)


x_train has shape (60000, 784), data type float64 and range (0.0, 1.0)
y_train has shape (60000,) and data type uint8
The shape of x_test is (10000, 784)
The shape of y_test is (10000,)
After one-hot coding, y_train has shape (60000, 10) and data type int32



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Training set output types (tf.float64, tf.int32)
Training set output shapes (TensorShape([Dimension(None), Dimension(784)]), TensorShape([Dimension(None), Dimension(10)]))


Validation set output types (tf.float64, tf.int32)
Validation set output shapes (TensorShape([Dimension(None), Dimension(784)]), TensorShape([Dimension(None), Dimension(10)]))


In [1]:
import tensorflow as tf
import numpy as np
"""
Prepare the dataset

Here I intentionally used tf.data.Dataset.from_tensor_slices 
to handle the input data, for two reasons:
1. To show that tf.keras can blend well with TensorFlow functions
2. For large dataset, tf.keras may benefit from tf.data.TFRecordDataset.
"""

batch_size = 100
batch_size_va = 600

graph = tf.Graph()
with graph.as_default():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    # for this small dataset, we can do pre-processing here.
    x_train = np.reshape(x_train / 255.0, (x_train.shape[0], -1))
    x_test = np.reshape(x_test / 255.0, (x_test.shape[0], -1))
    print('x_train has shape {}, data type {} and range {}'.format(
        x_train.shape, x_train.dtype, 
        (np.amin(x_train), np.amax(x_train))))
    print('y_train has shape {} and data type {}'.format(
        y_train.shape, y_train.dtype))
    print('The shape of x_test is {}'.format(x_test.shape))
    print('The shape of y_test is {}'.format(y_test.shape))
    
    # convert label to one-hot embedding as requested by Keras
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=10, dtype='int32')
    y_test = tf.keras.utils.to_categorical(y_test, num_classes=10, dtype='int32')
    print('After one-hot coding, y_train has shape {} and data type {}'.format(
        y_train.shape, y_train.dtype))
    
    # training set
    with tf.name_scope('data_tr'):
        dataset_tr = tf.data.Dataset.from_tensor_slices((x_train, y_train))
        # apply parser or pre-processing
        # if we have not scaled the data before, we can scale it here
        # dataset_tr = dataset_tr.map(lambda t: (t[0] / 255.0, t[1]))
        dataset_tr = dataset_tr.shuffle(1000).batch(batch_size).repeat()
        dataset_tr = dataset_tr.prefetch(tf.contrib.data.AUTOTUNE)
    print('Training set output types {}'.format(dataset_tr.output_types))
    print('Training set output shapes {}'.format(dataset_tr.output_shapes))
    
    # validation set
    with tf.name_scope('data_va'):
        dataset_va = tf.data.Dataset.from_tensor_slices((x_test, y_test))
        # apply parser or pre-processing
        # if we have not scaled the data before, we can scale it here
        # dataset_tr = dataset_tr.map(lambda t: (t[0] / 255.0, t[1]))
        dataset_va = dataset_va.batch(batch_size_va).repeat()
        dataset_va = dataset_va.prefetch(tf.contrib.data.AUTOTUNE)
    print('Validation set output types {}'.format(dataset_va.output_types))
    print('Validation set output shapes {}'.format(dataset_va.output_shapes))


x_train has shape (60000, 784), data type float64 and range (0.0, 1.0)
y_train has shape (60000,) and data type uint8
The shape of x_test is (10000, 784)
The shape of y_test is (10000,)
After one-hot coding, y_train has shape (60000, 10) and data type int32



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Training set output types (tf.float64, tf.int32)
Training set output shapes (TensorShape([Dimension(None), Dimension(784)]), TensorShape([Dimension(None), Dimension(10)]))


Validation set output types (tf.float64, tf.int32)
Validation set output shapes (TensorShape([Dimension(None), Dimension(784)]), TensorShape([Dimension(None), Dimension(10)]))


In [2]:
import tensorflow as tf
from tensorflow.python.client import timeline
"""
Define and train the model
"""
num_epoch = 1
summary_folder = 'C:/Users/richa/PycharmProjects/ai_mechatronics/Results/mnist/keras_log/'
do_trace = False

with graph.as_default():
    # define the model
    mdl = tf.keras.Sequential(name='MLP')
    # Adds a MLP
    mdl.add(
        tf.keras.layers.Dense(
            384, activation='relu', name='l1', 
            batch_input_shape=(None, 784)))
    mdl.add(tf.keras.layers.Dense(128, activation='relu', name='l2'))
    mdl.add(tf.keras.layers.Dense(10, activation='softmax', name='out'))
    """
    Alternatively, you may define the models as below, which is very much like 
    TensorFlow logic. 
    
    from tf.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Dropout
    
    # assume train and test data has been reshaped to (None, 28, 28, 1)
    inputs = Input(shape=(28, 28, 1))  # Input(shape=(784, )) for vector input
    hidden = Conv2D(32, (3, 3),activation='relu', padding='valid')(inputs)
    hidden = MaxPool2D(pool_size=(2, 2))(hidden)
    hidden = Conv2D(64, (3, 3), activation='relu')(hidden)
    hidden = MaxPool2D(pool_size=(2, 2))(hidden)
    hidden = Flatten()(hidden)
    hidden = Dense(512, activation='relu')(hidden)
    hidden = Dropout(0.5)(hidden)
    outputs = Dense(10, activation='softmax')(hidden)

    mdl = tf.keras.Model(inputs, outputs)
    """
    
    # configure profiling
    if do_trace:
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
        run_metadata = tf.RunMetadata()
    else:
        run_options = None
        run_metadata = None
    
    # configure training process
    mdl.compile(optimizer=tf.train.AdamOptimizer(0.01),
                loss='categorical_crossentropy', metrics=['accuracy'], 
                options=run_options, run_metadata=run_metadata)
    
    """
    Configure summary
    when write_graph=True, the event file size become quite large, 
    almost 1 GB in my case.
    """
    callbacks = [tf.keras.callbacks.TensorBoard(
        log_dir=summary_folder, histogram_freq=1, batch_size=batch_size, 
        write_graph=False, write_grads=True)]
    
    # fit the model
    if do_trace:
        """
        The reason I separate the tracing part from mdl.fit is that I observed
        sometimes mdl.fit yields an empty timeline.json file (size 1kb).
        
        Unfortunately, I have not figured out how to do tracing for multiple runs
        using purely keras.
        """
        for steps in range(2):
            mdl.train_on_batch(dataset_tr)
        # get the runtime statistics
        trace = timeline.Timeline(step_stats=run_metadata.step_stats)
        chrome_trace = trace.generate_chrome_trace_format()
        with open(summary_folder + 'timeline.json', 'w') as f:
            f.write(chrome_trace)
    else:
        steps_per_epoch = x_train.shape[0] // batch_size
        print('There are {} steps in one training epoch'.format(steps_per_epoch))
        mdl.fit(
            dataset_tr, epochs=num_epoch, steps_per_epoch=steps_per_epoch,
            validation_data=dataset_va, validation_steps=1, callbacks=callbacks)


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


In [10]:
import tensorflow as tf
"""
Save the model
"""
ckpt_folder = 'C:/Users/richa/PycharmProjects/ai_mechatronics/Results/mnist/keras_ckpt/'
ckpt_name = 'mnist_ckpt'
with graph.as_default():
    mdl.save_weights(ckpt_folder + ckpt_name)
